In [1]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms , models
from PIL import Image
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import os
from tqdm import tqdm

In [2]:
train = pd.read_csv('./input/data/train/train.csv')

In [3]:
class mydataset():
    def __init__(self):
        path=os.path.join(os.getcwd(),'input/data/train/images')
        imagelist=[]
        labellist=[]
        for folder in os.listdir(path):
            if folder.startswith('.'):
                continue
            id,gender,race,age = folder.split('_')
            age = int(age) - 18
            # 자료가 18세부터 60세까지의 자료만 있음 
            # 시작이 0부터 시작해서 똑같이 끝나지 않으면 오류생김
            for image in os.listdir(os.path.join(path, folder)):
                if image.startswith('.'):
                    continue
                mask,ext=image.split('.')
                imagelist.append(os.path.join(path,folder,image))
                labellist.append(age)
        self.imagelist=imagelist
        self.labellist=labellist
            

    def __len__(self):
        return len(self.imagelist)

    def __getitem__(self,idx):
        image_path = self.imagelist[idx]
        image = Image.open(image_path)
        transform = transforms.Compose([
            transforms.ToTensor()            
        ])
        image = transform(image)
        label = self.labellist[idx]
        return image,label
    


In [4]:
dataset=mydataset()
dataset[0][0].size()


torch.Size([3, 512, 384])

In [5]:
dataset=mydataset()
split=torch.utils.data.random_split(dataset, [int(18900*0.8), int(18900*0.2)])
trainset,valset = split[0],split[1]

In [6]:
train_loader = DataLoader(
    trainset,
    batch_size=256,
    num_workers=0,
    shuffle=True
)

val_loader = DataLoader(
    valset,
    batch_size=256,
    num_workers=0,
    shuffle=False
)


In [7]:
class mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad_(False)
        self.model.fc=nn.Linear(512,43)
                
    def forward(self, x):
        self.x = x
        self.x=self.model(self.x)
        return self.x

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model=mymodel().to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [10]:
epochs=10
batch_size=256
acc = 0
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    for i, data in tqdm(enumerate(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.cuda() , labels.cuda()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=-1)
        correct += (preds==labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data
    print('[%d, %d] loss: %.3f' %
      (epoch + 1, i + 1, running_loss / len(trainset)))
    running_loss = 0.0
    print(correct / len(trainset)*100)

print('Finished Training')

60it [01:31,  1.53s/it]


[1, 60] loss: 0.013
21.607142857142858


60it [01:30,  1.51s/it]


[2, 60] loss: 0.009
31.931216931216934


60it [01:31,  1.52s/it]


...

[8, 60] loss: 0.007
45.535714285714285


60it [01:30,  1.51s/it]


[9, 60] loss: 0.006
49.107142857142854


60it [01:30,  1.51s/it]

[10, 60] loss: 0.006
49.404761904761905
Finished Training


In [13]:
with torch.no_grad():
    correct=0
    model.eval()
    for val_batch in tqdm(val_loader):
        inputs, labels = val_batch
        print(inputs.size())
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=-1)
        correct += (preds==labels).sum().item()
    print(correct / len(valset)*100)


  0%|          | 0/15 [00:00<?, ?it/s]

torch.Size([256, 3, 512, 384])


  7%|▋         | 1/15 [00:01<00:22,  1.59s/it]

torch.Size([256, 3, 512, 384])


 13%|█▎        | 2/15 [00:03<00:20,  1.56s/it]

...

torch.Size([256, 3, 512, 384])


 93%|█████████▎| 14/15 [00:21<00:01,  1.56s/it]

torch.Size([196, 3, 512, 384])


100%|██████████| 15/15 [00:23<00:00,  1.54s/it]

38.17460317460317


#### 실제 사진으로 한장씩 확인해 보기

In [16]:
image = Image.open(os.path.join(os.getcwd()+'/20220306_085949.jpg'))
trans = transforms.Compose([
transforms.Resize((512,384)), # 학습에 사용한 사진의 사이즈로 바꾸기
transforms.ToTensor()])
image = trans(image)
image = image.unsqueeze(0) # 모양이 달라서 안돌아가기 때문에 3-dim에서 4-dim으로 바꾸기
image = image.to(device) # model이 cuda에 있기 때문에 image를 cuda로 보내야한다

with torch.no_grad():
    
    out = model(image)


# print(torch.argmax(out,1)+18) # 숫자를 맞추기 위해 18을 뺏기 때문에 다시 더해준다
print(out)

tensor([[ 4.6623, 10.5897, 10.3963,  2.8233,  1.9469,  9.0538,  5.4880,  5.5218,
          0.6536, -1.0104,  4.7255, -0.8707,  1.5600, -4.9228, -4.1081, -5.4867,
         -3.4603, -2.7632, -2.4701, -1.4610,  1.8224, -6.0496,  3.3955, -3.1493,
          1.5989,  2.8660,  0.3367, -0.9978, -7.3849,  2.3463,  0.1550, -1.2434,
          5.4339,  3.2828,  3.4053,  4.8501,  1.3520,  6.8891,  6.7978,  6.3045,
          5.7287,  8.8247,  6.8066]], device='cuda:0')
